<h1><font color="#113D68" size=6>Deep Learning con Python y Keras</font></h1>

<h1><font color="#113D68" size=5>Parte 3. Multilayer Perceptron</font></h1>

<h1><font color="#113D68" size=4>7. Proyecto de regresión</font></h1>

<br><br>
<div style="text-align: right">
<font color="#113D68" size=3>Manuel Castillo Cara</font><br>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [0. Contexto](#section0)
* [1. Topología de linea base](#section1)
* [2. Optimizar el rendimiento con procesamiento de datos](#section2)
* [3. Ajuste de la toplogía](#section3)
    * [3.1. Evaluar una topología más profunda](#section3.1)
    * [3.2. Evaluar una topología más grande](#section3.2)

---
<a id="section0"></a>
# <font color="#004D7F" size=6> 0. Contexto</font>

En este tutorial del proyecto, descubrirá cómo desarrollar y evaluar modelos de redes neuronales utilizando Keras para un problema de regresión. Después de completar este tutorial paso a paso, sabrá:
* Cómo cargar un conjunto de datos CSV y ponerlo a disposición de Keras.
* Cómo crear un modelo de red neuronal con Keras para un problema de regresión.
* Cómo utilizar scikit-learn con Keras para evaluar modelos mediante validación cruzada.
* Cómo realizar la preparación de datos para mejorar la habilidad con los modelos de Keras.
* Cómo ajustar la topología de red de modelos con Keras.

In [1]:
import tensorflow as tf
# Eliminar warning
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

2023-09-04 18:43:24.539597: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section1"></a>
# <font color="#004D7F" size=6>1. Topología de linea base</font>

En esta sección crearemos un modelo de red neuronal de referencia para el problema de regresión. Comencemos importando todas las funciones y objetos que necesitaremos para este tutorial.

Posteriormente, cargamos nuestro conjunto de datos desde un archivo en el directorio local. Luego, podemos dividir los atributos de entrada (X) y salida (Y) para que sean más fáciles de modelar con Keras y scikit-learn.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Más información sobre el dataset [Boston House Price](http://lib.stat.cmu.edu/datasets/boston)

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Nota: Puede ser que no le vaya con `delim_whitespace`. En ese caso que utilicen la siguiente sentencia:

`dataframe = pd.read_csv("Datasets/housing.csv", sep='\s+', header=None)`

In [1]:
# Binary Classification with Sonar Dataset: Baseline
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
# load dataset
dataframe = pd.read_csv("Datasets/housing.csv", delim_whitespace=True, header=None)

dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:13]
Y = dataset[:,13]

/Users/manwest/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/var/folders/9h/d3b8zc6j5019cyvmb_h929xm0000gn/T/ipykernel_6822/841839247.py:9: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dataframe = pd.read_csv("Datasets/housing.csv", delim_whitespace=True, header=None)


Ahora estamos listos para crear nuestro modelo de red neuronal usando Keras. Vamos a utilizar scikit-learn para evaluar el modelo mediante la validación cruzada. 

Para usar modelos de Keras con scikit-learn, debemos usar el contenedor `KerasClassifier`. Esta clase toma una función que crea y devuelve nuestro modelo de red neuronal. También toma argumentos que pasará a la llamada a `fit()` como el número de épocas y el tamaño del batch. 

Comencemos por definir la función que crea nuestro modelo de línea de base. 
1. Es un modelo simple que tiene una sola capa oculta completamente conectada con el mismo número de neuronas que los atributos de entrada (13). 
2. La red utiliza buenas prácticas como la función de activación ReLu para la capa oculta. 
    * No se utiliza ninguna función de activación para la capa de salida porque es un problema de regresión y estamos interesados en predecir valores numéricos directamente sin transformar.
3. Se utiliza el eficiente algoritmo de optimización ADAM y se optimiza una función de pérdida de error cuadrático medio. Es una métrica deseable porque al sacar la raíz cuadrada de un valor de error nos da un resultado que podemos entender directamente en el contexto del problema con las unidades en miles de dólares.
4. El objeto Wrapper de Keras para usar en scikit-learn como estimador de regresión se llama `KerasRegressor`. Creamos una instancia y le pasamos tanto el nombre de la función para crear el modelo de red neuronal como algunos parámetros para pasar a la función `fit()` del modelo, como el número de épocas y el tamaño de batchs.
5. El paso final es evaluar este modelo de línea de base. Usaremos una validación cruzada de 10.

Tenga en cuenta que el error cuadrático medio es negativo porque scikit-learn invierte de modo que la métrica se maximiza en lugar de minimizar. Puede ignorar el signo del resultado.

In [2]:
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim=13, activation='relu'))
    model.add(Dense(1))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
# evaluate model
estimator = KerasRegressor(model=baseline_model, epochs=50, batch_size=5, verbose=0)
kfold = KFold(n_splits=5)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

/Users/manwest/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-09-14 17:27:00.106430: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-09-14 17:27:00.106451: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 36.00 GB
2024-09-14 17:27:00.106455: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 13.50 GB
2024-09-14 17:27:00.106468: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-14 17:27:00.106479: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271]

Baseline: -1.45 (3.44) MSE


---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section2"></a>
# <font color="#004D7F" size=6>2. Optimizar el rendimiento con procesamiento de datos</font>

Una preocupación importante con el conjunto de datos de precios de la vivienda de Boston es que todos los atributos de entrada varían en sus escalas porque miden diferentes cantidades. Casi siempre es una buena práctica preparar los datos antes de modelarlos utilizando un modelo de red neuronal. 

Podemos utilizar el marco de trabajo [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) para realizar la estandarización durante el proceso de evaluación del modelo, dentro de cada fold. Esto asegura que no haya fugas de datos de cada fold de validación cruzada del conjunto de pruebas en los datos de entrenamiento.

La ejecución del ejemplo proporciona un rendimiento mejorado sobre el modelo de línea base sin datos estandarizados, eliminando el error.

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(model=baseline_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

/Users/manwest/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/manwest/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/manwest/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(act

Una extensión adicional de esta sección sería aplicar de manera similar un cambio de escala a la variable de salida, como normalizarla al rango de 0 a 1 y usar una función de activación Sigmoide o similar en la capa de salida para reducir las predicciones de salida al mismo rango.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section3"></a>
# <font color="#004D7F" size=6>3. Ajuste de la toplogía</font>

Hay muchas preocupaciones que se pueden optimizar para un modelo de red neuronal. Quizás el punto de mayor apalancamiento es la estructura de la red en sí, incluida la cantidad de capas y la cantidad de neuronas en cada capa. En esta sección evaluaremos dos topologías en un esfuerzo por mejorar aún más el rendimiento del modelo. Examinaremos una topología de red tanto más profunda como más amplia.

<a id="section3.1"></a>
# <font color="#004D7F" size=5>3.1. Evaluar una topología más profunda</font>

Una forma de mejorar el rendimiento de una red neuronal es agregar más capas. Esto podría permitir que el modelo extraiga y recombine características de orden superior integradas en los datos. Esto es tan fácil como definir una nueva función que creará este modelo más profundo, copiado de nuestro modelo de referencia anterior. Luego podemos insertar una nueva línea después de la primera capa oculta. En este caso con aproximadamente la mitad del número de neuronas. Nuestra topología de red ahora se ve así:
```
    13 entradas -> [13 -> 6] -> 1 salida
```
Podemos evaluar esta topología de red de la misma manera que anteriormente, al mismo tiempo que usamos la estandarización del conjunto de datos que se demostró anteriormente para mejorar el rendimiento.

In [4]:
# define the model
def larger_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim=13, activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(model=larger_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: 0.41 (0.51) MSE


La ejecución de este modelo muestra una mejora adicional en el rendimiento de MSE.

<a id="section3.2"></a>
# <font color="#004D7F" size=5>3.2. Evaluar una topología más grande</font>

Otro enfoque para aumentar la capacidad de representación del modelo es crear una red más amplia. Nuevamente, todo lo que tenemos que hacer es definir una nueva función que cree nuestro modelo de red neuronal. Aquí, hemos aumentado el número de neuronas en la capa oculta en comparación con el modelo de línea de base de 13 a 20. La topología de nuestra red más amplia se puede resumir de la siguiente manera:

```
    13 entradas -> [20] -> 1 salida
```

Podemos evaluar la topología de la red más amplia utilizando el mismo esquema que el anterior.

In [9]:
# define wider model
def wider_model():
    # create model
    model = Sequential()
    model.add(Dense(20, input_dim=13, activation='relu'))
    model.add(Dense(1))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(model=wider_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Wider: 0.39 (0.50) MSE


La construcción del modelo ve una caída adicional en el error a aproximadamente 22 mil dólares cuadrados. Este no es un mal resultado para este problema.

Habría sido difícil adivinar que una red más amplia superaría a una red más profunda en este problema. Los resultados demuestran la importancia de las pruebas empíricas cuando se trata de desarrollar modelos de redes neuronales.

<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>